In [74]:
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [60]:
train_df = pd.read_csv('../../data/train_clear.csv')[['Q', 'label']]
train_df = train_df.dropna()
train_df.head(2)

,Q,label
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,angry
1,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,angry


In [61]:
with open('train_text.txt', 'w', encoding='utf-8') as f:
    for line in train_df['Q'].values:
        try:
            f.write(line+'\n')
        except TypeError as e:
            print(line, e)

In [62]:
with open('train_text.txt', 'r', encoding='utf-8') as f:
    train_text = f.read().split('\n')
print(train_text[0])

일은 왜 해도 해도 끝이 없을까? 화가 난다.


In [63]:
import sentencepiece as spm

input_path = '../../data/train_text.txt'
vocab_size = 32000
model_prefix = f'spm/spm_{vocab_size}'
spm.SentencePieceTrainer.train(input=input_path, model_prefix=model_prefix, vocab_size=vocab_size)

In [64]:
sp = spm.SentencePieceProcessor()
sp.Load(f'spm/spm_{vocab_size}.model')

True

In [65]:
encoded = []
for t in train_df['Q'].values:
    encoded.append(sp.encode_as_ids(t))
train_text = pad_sequences(encoded)
print(train_text[:5])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0    87    19    70    27
      6    27     6  2916  1608    25   101  2235     3]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0   116     9  6067    28
   4884     3  9704  1567   738   305     6   183     3]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0   187  1912   209  2335     7 17548    90  6501    1

In [66]:
train_df['label'].unique()

array(['angry', 'happy', 'insecure', 'embarrassed', 'sad', 'broken_heart'],
      dtype=object)

In [67]:
emotions = {
    'angry': 0,
    'happy': 1,
    'insecure': 2,
    'embarrassed': 3,
    'sad': 4,
    'broken_heart': 5
}
train_df['label'] = train_df['label'].apply(lambda x: emotions[x])
train_df['label'].value_counts()

label
2    26298
0    26000
5    25954
4    25814
3    24936
1    16947
Name: count, dtype: int64

In [68]:
train_labels = train_df['label'].values
train_labels[:2]

array([0, 0], dtype=int64)

In [73]:
train_set = tf.data.Dataset.from_tensor_slices((train_text, train_labels)).shuffle(10000).batch(32)
print(train_set)

<BatchDataset element_spec=(TensorSpec(shape=(None, 57), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [ ]:
class MyLSTM(Model):
    def __init__(self):
        super(MyLSTM, self).__init__()
